<b> 
    <font size="7">
        Computational Finance and FinTech <br><br>
        M.Sc. International Finance
    </font>
</b>
<br><br>
<img src="pics/HWR.png" width=400px>
<br><br>
<b>
    <font size="5"> 
        Prof. Dr. Natalie Packham <br>
        Berlin School of Economics and Law <br>
        Summer Term 2025
    </font>
</b>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Option-Pricing" data-toc-modified-id="Option-Pricing-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Option Pricing</a></span><ul class="toc-item"><li><span><a href="#Call-and-put-options" data-toc-modified-id="Call-and-put-options-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Call and put options</a></span></li><li><span><a href="#The-one-period-model" data-toc-modified-id="The-one-period-model-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>The one-period model</a></span></li><li><span><a href="#The-multi-period-model" data-toc-modified-id="The-multi-period-model-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>The multi-period model</a></span></li><li><span><a href="#The-Cox-Ross-Rubinstein-model" data-toc-modified-id="The-Cox-Ross-Rubinstein-model-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>The Cox-Ross-Rubinstein model</a></span></li><li><span><a href="#Towards-a-continuous-time-model:-Brownian-motion" data-toc-modified-id="Towards-a-continuous-time-model:-Brownian-motion-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Towards a continuous-time model: Brownian motion</a></span></li><li><span><a href="#The-Black-Scholes(-Merton)-model" data-toc-modified-id="The-Black-Scholes(-Merton)-model-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>The Black-Scholes(-Merton) model</a></span></li><li><span><a href="#The-Black-Scholes-formula" data-toc-modified-id="The-Black-Scholes-formula-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>The Black-Scholes formula</a></span></li></ul></li></ul></div>

In [ ]:
import math
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

In [ ]:
# function to draw one-period binomial trees
def graphTree(labels,arrow_labels,extra_labels=[]): 
    # Define the positions for the nodes
    x = [0, 1, 1]
    y = [0, 1, -1]
    alignment=['right', 'left', 'left']

    # Create the plot
    fig, ax = plt.subplots()
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-1.5, 1.5)
    plt.xlim(-0.5, 1.6)
    plt.gca().set_aspect(aspect=0.4)

    # Annotate the nodes
    for i, label in enumerate(labels):
        ax.text(x[i], y[i], label, fontsize=12, ha=alignment[i], color='mediumblue')
        if(len(extra_labels)>0):
            ax.text(x[i], y[i] - 0.2, extra_labels[i], fontsize=12, ha=alignment[i], color='darkred')
    
    # Draw arrows
    arrowprops = dict(facecolor='black', shrink=0.05, width=2, headwidth=10)

    ax.annotate('', xy=(1, 1), xytext=(0, 0), arrowprops=arrowprops)
    ax.annotate('', xy=(1, -1), xytext=(0, 0), arrowprops=arrowprops)

    # Annotate the arrows
    ax.text(0.5, 0.7, arrow_labels[0], fontsize=12, ha='left')
    ax.text(0.5, -0.4, arrow_labels[1], fontsize=12, ha='left')

    # Set the x-axis
    ax.set_xticks([0, 1])
    ax.set_xticklabels(['0', '1'])

    # Hide y-axis
    ax.yaxis.set_visible(False)

    # Add grid for better visualization
    ax.grid(True, linestyle='--')

    # Add title
    plt.title('One-Period Binomial Tree')

    # Show the plot
    plt.show()
    
#graphTree([r'$S_0 = 100$', r'$S_1 = s \cdot 120$', r'$S_1 = s \cdot 80$'],[r'$0.6$', r'$0.4$'],['Option value=?', 'Option payoff: 15', 'Option payoff: 0'])

# Option Pricing

### Option Pricing

* This session is _not_ directly related to a chapter in __Py4Fi__.
* An __option__, also called __contingent claim__, is a financial derivative whose payoff is a (non-linear) function of an underlying asset's value.
* The most important and liquidly traded contingent claims are __call__ and __put options__. 
* Option prices are determined by assuming that markets are free of __arbitrage__.
* We cover standard option pricing techniques: the __binomial tree model__ and the __Black-Scholes(-Merton) model__.

## Call and put options

* In the following, $(S_t)_{t\geq 0}$ denotes the price process of the underlying asset. 
* A __Euopean call option__ with __exercise / strike price__ $K$ and maturity $T$ has payoff $X=\max(S_T-K,0)=(S_T-K)^+$.
* The holder of the option has the right, but not the obligation, to buy the stock at time $T$ at price $K$.
* A __European put option__ with __exercise / strike price__ $K$ and maturity $T$ has payoff $X=\max(K-S_T,0)=(K-S_T)^+$.

In [ ]:
x = np.linspace(0,200)
k=100; # strike price
plt.figure(figsize=(12, 3))
plt.subplot(121)
plt.plot(x,(x>k)*(x-k), lw=1.5)
plt.xlabel('$S_T$')
plt.title('call option payoff')
plt.subplot(122)
plt.plot(x, (x<k)*(k-x), lw=1.5)
plt.xlabel('$S_T$')
plt.title('put option payoff');

### Option valuation
* At first sight, it might be seem impossible to determine a price for an option. 
* It turns out that under some general conditions and assumptions, unique prices for contingent claims can be determined!
* The main assumption is that markets are __free of arbitrage__.
* A powerful result, under some further assumptions, then shows that the payoff of a contingent claim can be __replicated__ with a dynamic trading strategy in a bond and the underlying asset. 
* A rigorous derivation of this result is beyond the scope of the course, but we will sketch the main ingredients. 

Discuss: Is the assumption of __absence of arbitrage__ plausible in financial markets? 


### Option valuation - Roadmap
* We begin by setting up a model for a financial market consisting of a __riskless asset__, such as a __bond__ or a __money market account__ and a __risky asset__ e.g. a share of __stock__.
* The bond / money market account accrues at a constant deterministic interest rate, the __risk-free rate__. 
* The stock price process is modelled as a __stochastic process__, where, at every point in time the current price and the price history are known, but future prices are unknown. 

### Option valuation - Roadmap
* First, we consider a __discrete-time model__, where the stock price evolves in a __binomial tree__. 
* It turns out that the payoffs of contingent claims can be __replicated__ by a dynamic trading strategy in the stock and the bond.
* By a no-arbitrage argument, the price of the contingent claim must equal the cost setting up the replicating strategy.  
* Taking the time step length $\Delta t$ to the limit of $0$, yields a __continuous-time model__, the __Black-Scholes(-Merton) model__. 
* This gives a closed-form solution for call and put option prices, the famous __Black-Scholes formula__. 

## The one-period model 
* Example: 
    * Suppose you want to buy a __call option__ on a stock. 
    * The option matures in one year and has a strike price of 105. 
    * The current stock price is 100, and it is known that the stock price either increases or decreases by 20% within one year. 
    * The risk-free interest rate is 5% (discrete compounding).  
* Questions:
    * What is the __value__ of this option? (Pricing)
    * How can you __hedge__ this option? (Hedging)

### The one-period model 
* There are two points in time: $t\in \{0,1\}$
* Market consists of bond and stock
* Bond price is deterministic: $B_0=1$ and $B_1=1+r$
* Stock price $S_t$ is a stochastic process with $S_0=s$ and $S_1=s\cdot u$ with probability $p_u$ and $S_1=s\cdot d$ with probability $p_d$. 
* We also write $S_1=S_0\cdot Z$ where $Z$ is a random variable taking values in $\{u,d\}$. 

In [ ]:
graphTree([r'$S_0 = s$', r'$S_1 = s \cdot u$', r'$S_1 = s \cdot d$'],[r'$p_u$', r'$p_d$'])

### The one-period model
* Assumptions: 
    * $s, u, d, p_u, p_d$ are known
    * $d<u$
    * $p_u+p_d=1$
    
* A __portfolio__ is a vector $(x,y)$, where 
    * $x$ denotes the number bonds held and 
    * $y$ denotes the number of stocks held. 
    

### The one-period model
* Market assumptions:
    * Short positions and arbitrary holdings are allowed ($(x,y)\in \mathbb R^2$).
    * There are no bid-ask spreads.
    * There are no transaction costs.
    * The market is completely liquid, i.e., it is always possible to buy and sell unlimited quantities in the market.

### Portfolios
* Value process:
$V_t = x B_t + y S_t$, $t\in 0,1$.
* Equivalently:
\begin{align*}
 V_0&=x + y s\\
 V_1 &= x(1+r) + y S_0\cdot Z
 \end{align*}



### Arbitrage
* An __arbitrage portfolio__ or __arbitrage strategy__ is a portfolio $(x,y)$ with 

 
\begin{align*}
V_0&=0\\
\mathbb P(V_1\geq 0)&=1\\
\mathbb P(V_1>0)&>0
\end{align*}


* In words: An arbitrage opportunity is the possibility of generating at zero cost today a payoff at some time point in the future that is nonnegative with certainty and positive with positive probability.


### Arbitrage
* The one-period model is free of arbitrage if and only if $$d < 1+r < u.$$ 
* Interpretation: return on stock is not allowed to dominate bond, and vice versa.


### Example
* Consider a market with 
    * risk-free interest rate $r=5\%$, 
    * $S_0=100$, $u=1.2$, $d=0.8$, 
    * $p_u=0.6$, $p_d=0.4$.
* The market is free of arbitrage since $0.8 < 1.05 < 1.2$. 



### Example - Replicating a call option
* Now consider a call option with strike price $K=105$: 

In [ ]:
graphTree([r'$S_0 = 100$', r'$S_1 = s \cdot 120$', r'$S_1 = s \cdot 80$'],[r'$0.6$', r'$0.4$'],\
          ['Option value: ?', 'Option payoff: 15', 'Option payoff: 0'])

* We would like to build a portfolio satisfying: 
 
\begin{align*}
x (1+r) + y S_0\cdot u &= 15\\
 x (1+r) + y S_0\cdot d &= 0
 \end{align*}

### Example - replicating a call option
* Plugging in the numbers for $B_t, S_0\cdot u$ and $S_0\cdot d$ gives


\begin{align*}
        x \cdot 1.05 + 120 y &= 15\\
        x \cdot 1.05 + \phantom{1}80 y&=0.
      \end{align*}


* The solution to this system of linear equations is


\begin{equation*}
       y=\frac{15}{40} =\frac{3}{8}=0.375
       \quad\text{ and }\quad 
      x=-\frac{30}{1.05}=-\frac{30}{21/20} = -\frac{200}{7}
        =-28.57.
      \end{equation*}

      
* We have found that the portfolio $(-28.57, 0.375)$ __replicates__ the option payoff, that is, $V_1$ is idential to the option payoff. 

* The value of the portfolio today is $V_0 = -28.57 + 0.375 \cdot 100 = 8.93$. 


### Example - replicating a call option
* $8.93$ is the __only__ price for the option that is  consistent with the no-arbitrage principle:
* If the option could  be traded at a $\begin{Bmatrix}
\text{higher}\\
\text{lower}
\end{Bmatrix}$ price, then one could create a riskless profit by 
$\begin{Bmatrix}
\text{selling}\\
\text{buying}
\end{Bmatrix}$ the option and 
$\begin{Bmatrix}
\text{buying}\\
\text{selling}
\end{Bmatrix}$ 
the replicating portfolio.

In [ ]:
graphTree([r'$S_0 = 100$', r'$S_1 = s \cdot 120$', r'$S_1 = s \cdot 80$'],[r'$0.6$', r'$0.4$'],\
          ['Option value: 8.93', 'Option payoff: 15', 'Option payoff: 0'])

### Remarks
* The portfolio replicating the option payoff is called the __replicating portfolio__.
* The option price is called the __no-arbitrage price__ of the option. 
* Adding an option to the market that trades at its no-arbitrage price implies that the market remains free of arbitrage. 
* A buyer of the option can __hedge__ the exposure by entering into __minus__ the replicating portfolio.
* The probabilities $p_u, p_d$ did not enter into the calculation of the price. 


### Arbitrary contingent claims
* Definition: 
A __contingent claim (financial derivative)__ is any random variable $X$ of the form $X=\Phi(Z)$, where $Z$ is the random variable driving the stock price process. 

* $\Phi$ is called the __contract function__.


In [ ]:
graphTree([r'$S_0 = 100$', r'$S_1 = s \cdot u$', r'$S_1 = s \cdot d$'],[r'$p_u$', r'$p_d$'],\
          ['', 'Option payoff: ' + r'$\Phi(u)$', 'Option payoff: ' + r'$\Phi(d)$'])

### Arbitrary contingent claims
* __Proposition__: Suppose that a claim $X$ can be replicated with a portfolio $(x,y)$. Then any price of $X$ at $t=0$ other than $V_0^{(x,y)}$ leads to an arbitrage opportunity.
* __Proposition__: Assume that the binomial model is free of arbitrage. Then the market is __complete__, i.e., all contingent claims can be replicated. 


### Arbitrary contingent claims
* Proof: 
    * Fixing an arbitrary claim $X$ with contract function $\Phi$, we require a solution $(x,y)$ to the system of linear equations,
    \begin{align*}
    (1+r) x + S_0\,uy &= \Phi(u),\\
    (1+r) x + S_0\,dy &= \Phi(d).
    \end{align*}
    
    &nbsp;
    * Since $u>d$, the linear system has a unique solution, given by  
    &nbsp;
    
    \begin{align}
        x&= \frac{u\Phi(d)-d\Phi(u)} {(1+r) (u-d)}\label{eq:2}&
    y&= \frac{1}{S_0} \cdot \frac{\Phi(u)-\Phi(d)}{u-d}.
      \end{align}
   

### Risk-neutral pricing
* Recall that the market is free of arbitrage if and only if $d<1+r<u$. 
* This is equivalent to saying that: there exist $q_u, q_d> 0$, with $q_u+q_d=1$, such that


\begin{equation*}
      1+r = q_u\cdot u + q_d \cdot d. 
\end{equation*}
    
* $q_u,q_d$ can be interpreted as a __probability measure__ $\mathbb Q$ with

    \begin{equation*}
      \mathbb Q(Z=u)=q_u, \qquad \mathbb Q(Z=d)=q_d.
    \end{equation*}



### Risk-neutral pricing
* By definition, we find 
&nbsp;

 \begin{equation*}
 \frac{1}{1+r} \mathbb E^\mathbb Q[S_1] = \frac{1}{1+r}
      \left[q_u S_0\, u + q_d S_0\, d\right] = \frac{1}{1+r}\cdot S_0 (1+r) =
      S_0.
    \end{equation*}

* The probabilities $q_u, q_d$ are called __risk-neutral probabilities__ if the following condition holds:

    \begin{equation*}
      S_0=\frac{1}{1+r} \mathbb E^\mathbb Q[S_1]. 
    \end{equation*}


### Example
* The risk-neutral probabilities in the example are given by
&nbsp;

 \begin{align*}
        q_u &= \frac{1.05-0.8}{1.2-0.8} = \frac{0.25}{0.4} =
              \frac{5}{8} = 0.625\\[5pt]
        q_d &= 1-q_u = 0.375
      \end{align*}


* The no-arbitrage price is calculated as
    \begin{equation*}
        \text{Price}(\max(S_1-105,0)) := \frac{1}{1.05} \left(0.625\cdot 15 +   0.375\cdot 0\right) = 8.93. 
      \end{equation*}


### Risk-neutral pricing
* We re-state the "no-arbitrage" proposition in a way that is not specific to the one-period model. 
* __Proposition (No-arbitrage, First Fundamental Theorem)__: The market model is arbitrage free if and only if there exists a risk-neutral measure $\mathbb Q$.

### Risk-neutral pricing
* __Proposition__: For the one-period model, the risk-neutral probabilities are given by 

    \begin{align}
      q_u &= \frac{(1+r)-d}{u-d}\label{eq:5}\\[5pt]
      q_d&= 1-q_u = \frac{u-(1+r)}{u-d}. \label{eq:4}
    \end{align}


* __Proposition__: If the binomial model is free of arbitrage, then the arbitrage-free price of a contingent claim $X$ is given by

    \begin{equation*}
      \text{Price}(X) = \frac{1}{1+r} \mathbb E^\mathbb Q [X].  
    \end{equation*}


### Risk-neutral probabilities
* Objective probabilities determine which events are possible and which are impossible.
* We compute the arbitrage free price of a financial derivative __as if__ we were living in a risk-neutral world.
* This does __not__ mean that we believe that we live in a risk-neutral world. 
* Rather, investors __do not receive a risk premium__ for holding contingent claims that can be entirely risk-managed by replication. 
* The valuation formula holds for all investors, regardless of their attitude towards risk.

## The multi-period model

* Discrete time, with time running from $t=0$ to $t=T$. 
* Market consists of two assets, a __bond__ and a __stock__
* The __bond price process__ denoted by $B_t$, is given by 

\begin{align*}
          B_{t+1}&= (1+r)\, B_t\\
          B_0&= 1,
\end{align*}
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
with $r$ the interest rate (a simple period rate).

### The multi-period model
* The dynamics of the __stock process__, 
    denoted by $S_t$, are 

\begin{align*}
          S_{t+1} &= S_t\cdot Z_t,\\
          S_0 &=s,
\end{align*}

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
where $Z_0, \ldots, Z_{T-1}$ are i.i.d. random variables with $$\mathbb P(Z_t=u)=p_u, \qquad \mathbb P(Z_t=d)=p_d.$$

![Binomial tree](pics/binomial5.png)




### Portfolios
* __Definition__: A __portfolio strategy__ is a stochastic process

 \begin{equation*}
\{(x_t,y_t), \quad t=1,\ldots, T\},
\end{equation*}

such that $h_t$ is a function of $S_0, S_1, \ldots, S_{t-1}$.
* The __value process__ corresponding to portfolio $(x,y)$ is defined by

\begin{equation*}
V_t^{(x,y)} = x_t(1+r) + y_t S_t. 
\end{equation*}

* $x_t$ is the amount of money invested at time $t-1$ and kept until time $t$.
* $y_t$ is the number of shares bought at time $t-1$ and kept until time $t$.
* At any point in time, the portfolio strategy can depend on all information available at that time...
* ... but of course, one cannot look into the  future.


### Portfolios
* __Definition__: A portfolio strategy $(x,y)$ is said to be __self-financing__ if the following condition holds for all $t=1, \ldots, T-1$,

    \begin{equation*}
      x_t(1+R) + y_t S_t = x_{t+1} + y_{t+1} S_t.
    \end{equation*}


* This expresses that, at each time $t$, the market value of the portfolio created at $t-1$ equals the market value of the portfolio created at $t$.
* In other words: no funds are injected or withdrawn from the portfolio strategy at times $t=1,\ldots, T-1$.
* The portfolio is merely __rebalanced__ at every time point, that is, the holdings in the risky asset and risk-free asset are changed subject to keeping the portfolio value constant. 


### Multi-period model
* Everything else - arbitrage, condition for absence of arbitrage, risk-neutral measure - stays the same. 
* One-period risk-neutral pricing formula is a conditional expectation:

    \begin{equation*}
      S_t=\frac{1}{1+r} \mathbb E^\mathbb Q[S_{t+1}|S_t].
    \end{equation*}


* __Proposition__: The multiperiod binomial model is complete, that is, every claim can be replicated by a self-financing portfolio.


### Multi-period model
* Break down multi-period model into one-period models: 

![Binomial tree](pics/binomial6.png)


* Price and replicate the claim backwards, step-by-step.



### Example
* Spot price $S_0=140$
* up and down move factors: $u=1.5$, $d=0.78571$
* interest rate $r=0.1$
* strike $K=160$
* maturity $T=2$

* First determine:
     * Risk-neutral probabilities: $q_u = 0.44$, $q_d=0.56$
     * Contract function: $$\Phi(S_2)=\max(S_2-K,0)$$

### Multi-period model

![Binomial tree](pics/binomial7.png)




### Multi-period model

![Binomial tree](pics/binomial8.png)




### Multi-period model

![Binomial tree](pics/binomial9.png)



### Multi-period model

* __Risk-neutral pricing__:
\begin{align*}
\text{Price}(X)&=\frac{1}{(1+r)^2} \mathbb E^\mathbb Q[\Phi(S_T)]\\[5pt]
&=\frac{1}{(1+r)^2} \mathbb E^\mathbb Q[q_u^2 \Phi(S_0\cdot u^2) + 2 q_u q_d \Phi(S_0\cdot u d) + q_d^2 \Phi(S_0\cdot d^2)] \\[5pt]
&= 26.84.
 \end{align*}




### Multi-period model
* The time-$T$ stock price outcomes follow a __binomial distribution__.
* The price of an option $X$ with contract function $\Phi$ is thus:
\begin{equation*}
\text{Price}(X) = \frac{1}{(1+r)^T}\cdot \sum_{k=0}^T \binom{T}{k}
    q_u^k\, q_d^{T-k} \Phi(s u^k\, d^{T-k}). 
  \end{equation*}
  
* Note: This formula applies only to European non-path dependent claims.

### Multi-period model
* Let's implement the binomial tree in Python and calculate the price from the example.

In [ ]:
import math

S0=140.
T=2
r=0.1
u=1.5
d=0.78571

In [ ]:
def calculate_tree():
    S=np.zeros((T+1, T+1))
    S[0,0]=S0
    z=1
    for t in range(1, T+1):
        for k in range(z): # number of up moves
            S[k+1,t] = S[k, t-1] * u
            S[k,t] = S[k,t-1] * d
        z = z + 1
    return np.flipud(S) # flip the result so that up moves are actually "up"

### Multi-period model

In [ ]:
np.set_printoptions(formatter = {'float':lambda  x: ' %6.2f ' % x})

In [ ]:
calculate_tree()

## The Cox-Ross-Rubinstein model
* The __Cox-Ross-Rubinstein model (CRR model)__, is a special case of the multiperiod binomial tree model.
* Fix a time interval $[0,T]$ and set $\Delta t=T/N$.
* Trading takes place at times

    \begin{equation*}
      0,\Delta t, 2\Delta t, \ldots,
      (N-1)\Delta t, N\Delta t=T. 
    \end{equation*}


* The bond price accrues continuously at rate $r$, so that $B_t=B_0\,\mathrm e^{r t}$ and $B_{n\Delta t}=B_{(n-1) \Delta t} \, \mathrm e^{r\Delta t}$, where $n=1,\ldots, N$.
* The risk of the stock price is expressed by the __volatility__ $\sigma$, the standard deviation of the stock price's log-return.
* The volatility for the time period $\Delta t$ is $\sigma\sqrt{\Delta t}$, yielding

    \begin{equation*}
      u= \mathrm e^{\sigma\sqrt{\Delta t}},\qquad
      d= 1/u = \mathrm e^{-\sigma \sqrt{\Delta t}}. 
    \end{equation*}


## Towards a continuous-time model: Brownian motion
* Taking $\Delta t\rightarrow 0$ gives the __Black-Scholes model__ in the continuous-time limit.
* Note that $T$ stays fixed, but the discrete-time grid is made finer and finer. 
* The binomial distribution associated with time-$T$ stock prices converges to a normal distribution (this is a consequence of the so-called Central Limit Theorem).  
* The up-down moves of the stock price path through the binomial tree converge to a continuous-time stochastic process called a __Brownian motion__. 
* This is a process with independent normally distributed increments. 

### Brownian motion
* A stochastic process $W=(W_t)_{t\geq 0}$ is a **Brownian motion** if
    * $W_0=0$ and $W$ has continuous paths,
    * $W$ has independent increments, that is, for $r<s\leq t<u$, the random variables $W_u-W_t$ and $W_s-W_r$ are independent,
    * For $s<t$, $W_t-W_s\sim\mathrm N(0,t-s)$, that is, the random variable $W_t-W_s$ is normally distributed with mean $0$ and variance $t-s$.
    


### Normal distribution

In [ ]:
import scipy.stats as scs
x=np.linspace(-5,5,100)
plt.plot(x,scs.norm.pdf(x));
plt.xlabel('x')
plt.ylabel('normal density');

### Brownian motion
* Some example paths: 

In [ ]:
import numpy.random as npr  

npr.seed(1235)
n=1000
z = npr.normal(size=(n,5))
w = np.cumsum(z,axis=0)
plt.plot(w);
plt.xlabel('time')
plt.title('Brownian motion paths');

### Bond price dynamics
* Continuous compounding at interest rate $r>0$ of a __bond__ or __money market account__ is governed by the __differential equation__:

\begin{equation*}
        d B_t = B_t r \, d t,\quad t\geq 0.
\end{equation*}


* The solution to the differential equation is given by

\begin{equation*}
        B_t = B_0\exp( r t), \quad t\geq 0.
\end{equation*}


### Stock price dynamics: Geometric Brownian motion
* Similarly, the __Black-Scholes model__ specifies the dynamics of a stock price as 

\begin{equation*}
         d S_t = S_t\, \mu\, d t + S_t\, \sigma\, d W_t.
\end{equation*}

* The solution of the differential equation turns out to be

\begin{equation*}
     S_t = S_0 \, \exp\left(\left(\mu-\frac{1}{2}\sigma^2\right)t + \sigma  W_t\right). 
\end{equation*}


### Geometric Brownian motion


In [ ]:
import numpy.random as npr  

npr.seed(1235)
n=250
m=500;
dt=1/n;
z2 = npr.normal(size=(n,m))
z = np.zeros((n+1,m))
z[1:]=z2
w = np.cumsum(z,axis=0)
s = np.exp(0.2 * np.sqrt(dt) * w)
det_part = np.exp((0.05-0.5*0.2**2)* dt *np.arange(0,n+1));
for i in range(0,m):
    s[:,i] = s[:,i] * det_part

### Geometric Brownian motion


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(np.arange(0,n+1)*dt,s);
plt.xlabel('time')
plt.title('Example stock price paths');
plt.subplot(122)
plt.hist(s[-1]);
plt.xlabel('$S_T$')
plt.title('frequencies of simulated stock prices');


## The Black-Scholes(-Merton) model
* Assume a frictionless financial market in which a bond and a stock are traded. 
    * (Assets are liquidly traded, there are no short-selling constraints, there are no transaction costs and there are no bid-ask-spreads.)
* In the __Black-Scholes(-Merton) model__ the dynamics of the bond price and the stock price are given by 
\begin{align*}
 d B_t &= r B_t\, d t \\ 
 d S_t &= \mu S_t\, d t + \sigma S_t\, d W_t,
 \end{align*}
where
    * $r,\mu, \sigma$ are deterministic constants,
    * $W=(W_t)_{t\geq 0}$ is a Brownian motion,
    * $r$ is the __risk free rate__
    * $\mu$ is the __drift__,
    * $\sigma$ is the __volatility__.

### Risk-neutral measure
* Because the Black-Scholes model is the result of taking the CRR model to the continuous-time limit, all properties such as the existence of replicating strategies, market completeness, existence of a risk-neutral probability measure carry over. 
* The stock price process, under the real-world probability measure $\mathbb P$, evolves as

 \begin{equation*}
d S_t =\mu S_t\, d t + \sigma S_t\, d W_t, \quad t\geq 0. 
\end{equation*} 
* Under the __risk-neutral probability measure__ $\mathbb Q$, the stock price process is 
\begin{equation*}
d S_t = r S_t \, d t + \sigma S_t\, d \overline W_t,\quad t\geq 0,
\end{equation*}
where $(\overline W_t)_{t\geq 0}$ is a Brownian motion under $\mathbb Q$. 
* Likewise, the bond price process evolves as 
 \begin{equation*}
d B_t = r B_t  \, d t,\quad t\geq 0.
\end{equation*} 


### Risk-neutral pricing

* __Proposition__: The __no arbitrage price__ of a contingent claim $Y=f(S_T)$ is given by $\mathrm e^{-rT} \mathbb E Y$, where expectation is taken under the risk-neutral measure. 
* Option pricing boils down to taking expectations under the risk-neutral measure!


## The Black-Scholes formula

* The famous Black-Scholes formula can be proved by calculating the expected value of a call option payoff. 

* __Theorem (Black-Scholes formula)__: 
    In the Black-Scholes model, the price of a European call option with parameters $K$, $T$, $r$, $\sigma$ and $S_0$ is given by

\begin{equation*}
S_0\, \mathrm N(d_{+}) - \mathrm e^{-r T} K\,\mathrm N(d_{-}),
\end{equation*} 
&nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
with
\begin{equation*}
d_{\pm} = \frac{\ln(S_0/K) +(r\pm \frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},
\end{equation*} 
&nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;
where $\mathrm N(x)$ is the cumulative distribution function of the standard normal distribution.


### Hedging in the Black-Scholes model
* At any point in time $t \leq T$, the **replicating portfolio** of a European call option maturing at $T$ with time-$t$ price $F(t,S_t)$ consists of

     * $\displaystyle\frac{\partial F(t,S_t)}{\partial S}={\mathrm N}(d_{t,+})$ units of stock and
     
     * $F(t,S_t)-\displaystyle\frac{\partial F(t,S_t)} {\partial S}\, S_t=-{\mathrm e}^{-r(T-t)} K {\mathrm N}(d_{t,-})$ units of the bond.  
     

### Hedging in the Black-Scholes model

* The partial derivative is the **sensitivity** of the call option with respect to the underlying asset. 
* Small movements $\Delta S_t$ in either direction in the asset price change the option price by approximately $\displaystyle \frac{\partial F(t,S_t)}{\partial S}\, \Delta S_t$:<br>


![Hedging](pics/hedge.png)


### Greeks
* The **sensitivities** of an option position are called **"Greeks"**.
* Greeks: The sensitivities of a contingent claim's price process $F(t,S_t)$
    with respect to the input parameters are:

\begin{align*}
      \text{Delta:}\qquad \Delta_t &= \frac{\partial F(t,S_t)}{\partial S}\\[5pt]
      \text{Gamma:} \qquad \Gamma_t &= \frac{\partial^2
                                      F(t,S_t)}{\partial S^2}
                                     & \text{(sensitivity of Delta)}\\[5pt]
      \text{Theta:} \qquad \Theta_t &= \frac{\partial F(t,S_t)}{\partial t}\\[5pt]
      \text{Vega:}\qquad \nu_t &= \frac{\partial F(t,S_t)}{\partial
                                 \sigma} & \text{("model risk")}\\[5pt]
      \text{Rho:} \qquad\rho_t &= \frac{\partial F(t,S_t)}{\partial r}
\end{align*}

### Greeks
* In the case of a call option, the Greeks are given by:


\begin{align*}
      \text{Delta:} \qquad \Delta_t &= {\mathrm N}(d_{t,+})\\[10pt]
      \text{Gamma:} \qquad \Gamma_t &= \frac{{\mathrm N}^\prime(d_{t,+})}{S_0
        \sigma
        \sqrt{T}}\\[10pt]
      \text{Theta:} \qquad \Theta_t &= - \frac{S_0
        {\mathrm N}^\prime(d_{t,+})\sigma}{2 \sqrt{T}} - r K e^{-rT} {\mathrm N}(d_{t,-})\\[10pt]
      \text{Vega:} \qquad \nu_t &= S_0 \sqrt{T} {\mathrm N}^\prime(d_{t,+})\\[10pt]
      \text{Rho: } \qquad \rho_t &= K e^{-rT} {\mathrm N}(d_{t,-}) T 
\end{align*}


### Greeks
* Call option prices and greeks: 

![Greeks](pics/greeks.png)

